<a href="https://colab.research.google.com/github/steelpipe75/kagglebook-for-colab/blob/master/ch07/ch07-01-stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

In [2]:
import importlib
import sys
import subprocess

# Google Colab 上で実行しているかどうかを判断するフラグ
ON_COLAB = "google.colab" in sys.modules
print(f"ON_COLAB: {ON_COLAB}")

if ON_COLAB:
    USE_GIT = True # Gitを使う
    # USE_GIT = False # Gitを使わない

    import sys
    import os

    print(f"USE_GIT: {USE_GIT}")
    if USE_GIT:
        !git clone https://github.com/ghmagazine/kagglebook.git

        sys.path.append('/content/kagglebook/ch07')
    else:
        # Google Drive にマウントする
        drive = importlib.import_module("google.colab.drive")
        drive.mount("/content/drive/")

        colab_dir = "/content/drive/MyDrive/kagglebook/" # データ置き場
        sys.path.append(os.path.join(colab_dir, 'ch07'))

ON_COLAB: True
USE_GIT: True
fatal: destination path 'kagglebook' already exists and is not an empty directory.


train_xは学習データ、train_yは目的変数、test_xはテストデータ
pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

In [3]:
if ON_COLAB:
    if USE_GIT:
        train = pd.read_csv('/content/kagglebook/input/sample-data/train_preprocessed.csv')
    else:
        train = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/train_preprocessed.csv'))
else:
    train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
if ON_COLAB:
    if USE_GIT:
        test_x = pd.read_csv('/content/kagglebook/input/sample-data/test_preprocessed.csv')
    else:
        test_x = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/test_preprocessed.csv'))
else:
    test_x = pd.read_csv('../input/sample-data/test_preprocessed.csv')

In [4]:
# neural net用のデータ
if ON_COLAB:
    if USE_GIT:
        train_nn = pd.read_csv('/content/kagglebook/input/sample-data/train_preprocessed_onehot.csv')
    else:
        train_nn = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/train_preprocessed_onehot.csv'))
else:
    train_nn = pd.read_csv('../input/sample-data/train_preprocessed_onehot.csv')
train_x_nn = train_nn.drop(['target'], axis=1)
train_y_nn = train_nn['target']
if ON_COLAB:
    if USE_GIT:
        test_x_nn = pd.read_csv('/content/kagglebook/input/sample-data/test_preprocessed_onehot.csv')
    else:
        test_x_nn = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/test_preprocessed_onehot.csv'))
else:
    test_x_nn = pd.read_csv('../input/sample-data/test_preprocessed_onehot.csv')

In [5]:
# ---------------------------------
# スタッキング
# ----------------------------------
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

models.pyにModel1Xgb, Model1NN, Model2Linearを定義しているものとする
各クラスは、fitで学習し、predictで予測値の確率を出力する

In [6]:
from models import Model1Xgb, Model1NN, Model2Linear

In [7]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [8]:
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1a = Model1Xgb()
pred_train_1a, pred_test_1a = predict_cv(model_1a, train_x, train_y, test_x)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:12:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-logloss:0.41663	eval-logloss:0.43550
[1]	train-logloss:0.37126	eval-logloss:0.39889
[2]	train-logloss:0.33889	eval-logloss:0.37205
[3]	train-logloss:0.31320	eval-logloss:0.35606
[4]	train-logloss:0.29062	eval-logloss:0.33709
[5]	train-logloss:0.27315	eval-logloss:0.32549
[6]	train-logloss:0.25735	eval-logloss:0.31328
[7]	train-logloss:0.24299	eval-logloss:0.30434
[8]	train-logloss:0.22850	eval-logloss:0.29755
[9]	train-logloss:0.21984	eval-logloss:0.29490
[0]	train-logloss:0.41964	eval-logloss:0.41762
[1]	train-logloss:0.37576	eval-logloss:0.38620
[2]	train-logloss:0.34343	eval-logloss:0.36018
[3]	train-logloss:0.31776	eval-logloss:0.34322
[4]	train-logloss:0.29396	eval-logloss:0.32650
[5]	train-logloss:0.27618	eval-logloss:0.31509


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:13:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[6]	train-logloss:0.26377	eval-logloss:0.30590
[7]	train-logloss:0.24848	eval-logloss:0.29695
[8]	train-logloss:0.23521	eval-logloss:0.28744
[9]	train-logloss:0.22311	eval-logloss:0.28122


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:13:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-logloss:0.41890	eval-logloss:0.43249
[1]	train-logloss:0.37293	eval-logloss:0.39662
[2]	train-logloss:0.34196	eval-logloss:0.37120
[3]	train-logloss:0.31459	eval-logloss:0.35191
[4]	train-logloss:0.29310	eval-logloss:0.33963
[5]	train-logloss:0.27737	eval-logloss:0.32667
[6]	train-logloss:0.25757	eval-logloss:0.31103
[7]	train-logloss:0.24559	eval-logloss:0.30324
[8]	train-logloss:0.23499	eval-logloss:0.29798
[9]	train-logloss:0.22137	eval-logloss:0.28779


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:13:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-logloss:0.41761	eval-logloss:0.43674
[1]	train-logloss:0.37038	eval-logloss:0.39970
[2]	train-logloss:0.33898	eval-logloss:0.37438
[3]	train-logloss:0.31250	eval-logloss:0.35629
[4]	train-logloss:0.28992	eval-logloss:0.34197
[5]	train-logloss:0.27194	eval-logloss:0.33111
[6]	train-logloss:0.25641	eval-logloss:0.32024
[7]	train-logloss:0.24382	eval-logloss:0.31233
[8]	train-logloss:0.23406	eval-logloss:0.30578
[9]	train-logloss:0.22478	eval-logloss:0.30087


In [9]:
model_1b = Model1NN()
pred_train_1b, pred_test_1b = predict_cv(model_1b, train_x_nn, train_y, test_x_nn)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.4762 - val_loss: 0.3825
Epoch 2/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.3687 - val_loss: 0.3743
Epoch 3/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.3368 - val_loss: 0.3656
Epoch 4/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.3076 - val_loss: 0.3553
Epoch 5/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2880 - val_loss: 0.3367
Epoch 6/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.2658 - val_loss: 0.3198
Epoch 7/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.2408 - val_loss: 0.2959
Epoch 8/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2092 - val_loss: 0.2944
Epoch 9/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1934 - val_loss: 0.2906
Epoch 10/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1795 - val_loss: 0.2966


AttributeError: 'Sequential' object has no attribute 'predict_proba'

In [ ]:
# 1層目のモデルの評価
print(f'logloss: {log_loss(train_y, pred_train_1a, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y, pred_train_1b, eps=1e-7):.4f}')

In [ ]:
# 予測値を特徴量としてデータフレームを作成
train_x_2 = pd.DataFrame({'pred_1a': pred_train_1a, 'pred_1b': pred_train_1b})
test_x_2 = pd.DataFrame({'pred_1a': pred_test_1a, 'pred_1b': pred_test_1b})

In [ ]:
# 2層目のモデル
# pred_train_2は、2層目のモデルの学習データのクロスバリデーションでの予測値
# pred_test_2は、2層目のモデルのテストデータの予測値
model_2 = Model2Linear()
pred_train_2, pred_test_2 = predict_cv(model_2, train_x_2, train_y, test_x_2)
print(f'logloss: {log_loss(train_y, pred_train_2, eps=1e-7):.4f}')